# Azure Video Analytics Demo

## Preprocessing video

## 

## Prerequisites

In [ ]:
! pip install azure-search-documents==11.6.0b4
! pip install openai python-dotenv azure-identity cohere azure-ai-vision-imageanalysis
! pip install azure-storage-blob
! pip install opencv-python pandas

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AIServicesVisionParameters,
    AIServicesVisionVectorizer,
    AIStudioModelCatalogName,
    AzureMachineLearningVectorizer,
    AzureOpenAIVectorizer,
    AzureOpenAIModelName,
    AzureOpenAIParameters,
    BlobIndexerDataToExtract,
    BlobIndexerParsingMode,
    CognitiveServicesAccountKey,
    DefaultCognitiveServicesAccount,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexerExecutionStatus,
    IndexingParameters,
    IndexingParametersConfiguration,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataIdentity,
    SearchIndexerDataSourceConnection,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    VisionVectorizeSkill
)
from azure.search.documents.models import (
    HybridCountAndFacetMode,
    HybridSearch,
    SearchScoreThreshold,
    VectorizableTextQuery,
    VectorizableImageBinaryQuery,
    VectorizableImageUrlQuery,
    VectorSimilarityThreshold,
)
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from IPython.display import Image, display, HTML
from openai import AzureOpenAI

In [ ]:
# Load environment variables
load_dotenv()

# Configuration
AZURE_AI_VISION_API_KEY = os.getenv("AZURE_AI_VISION_API_KEY")
AZURE_AI_VISION_ENDPOINT = os.getenv("AZURE_AI_VISION_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
BLOB_CONTAINER_NAME = os.getenv("BLOB_CONTAINER_NAME")
INDEX_NAME = "azure-ai-video-analytics"
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")

In [ ]:
# User-specified parameter
USE_AAD_FOR_SEARCH = False  # Set this to False to use API key for authentication

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=AZURE_SEARCH_ADMIN_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)

## Extracting frames from video

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, generate_blob_sas, BlobSasPermissions
import datetime

# Setting for Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)

def upload_to_blob(file_path, blob_name):
    blob_client = blob_service_client.get_blob_client(container=BLOB_CONTAINER_NAME, blob=blob_name)
    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    
    start_time = datetime.datetime.now(datetime.timezone.utc)
    expiry_time = start_time + datetime.timedelta(days=1)
    
    sas_token = generate_blob_sas(
        account_name=blob_client.account_name,
        container_name=blob_client.container_name,
        blob_name=blob_client.blob_name,
        account_key=blob_service_client.credential.account_key,
        permission=BlobSasPermissions(read=True),
        expiry=expiry_time,
        start=start_time
    )
    blob_url = f"https://{blob_client.account_name}.blob.core.windows.net/{blob_client.container_name}/{blob_client.blob_name}"
    blob_url_with_sas = f"{blob_url}?{sas_token}"
    return blob_url, blob_url_with_sas

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
#Note: This code sample requires OpenAI Python library version 1.0.0 or higher.
import os
from openai import AzureOpenAI
import json


def create_caption_by_gpt(image_url):
	print("Image URL:", image_url)
	print("Generating caption using GPT-4o model...")
	client = AzureOpenAI(
		azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
		api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
		api_version="2024-02-01"
	)

	system_message = """
	You are an excellent caption creator. You must create a detailed caption for the image below with a maximum of 100 characters.
	The created caption will be used to describe the image in the search index.
	"""

	message_text = [
		{"role":"system","content":system_message},
		{"role":"user","content": [
			{
				"type": "text",
				"content": f"Please provide a caption for the image."
			},
			{ 
				"type": "image_url",
				"image_url": {
					"url": image_url
				}
			}
		]}
	]


	completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		temperature=0,
		max_tokens=200,
		top_p=0.95,
		frequency_penalty=0,
		presence_penalty=0,
		stop=None
		)
	print(completion.choices[0].message.content)
	return completion.choices[0].message.content


In [ ]:
import cv2
import pandas as pd
import os

def extract_frames(video_path, output_dir, interval=5):
    # Get the video file name
    video_filename = os.path.basename(video_path)
    
    # Create the output directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Load the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    
    # List to save metadata
    metadata = []

    # Extract frames
    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = frame_number / fps
        if current_time % interval < 1.0 / fps:
            # Save the frame as an image
            frame_filename = f"{video_filename}_frame_{frame_number}.jpg"
            frame_filepath = os.path.join(output_dir, frame_filename)
            cv2.imwrite(frame_filepath, frame)
            
            # Upload the frame image to Blob storage
            blob_url, blob_url_with_sas = upload_to_blob(frame_filepath, frame_filename)
            
            # Generate a caption for the frame image
            caption = create_caption_by_gpt(blob_url_with_sas)
            
            # Save the metadata
            metadata.append({
                'video_filename': video_filename,
                'frame_filename': frame_filename,
                'frame_url': blob_url,
                'frame_number': frame_number,
                'timestamp': current_time,
                'caption': caption
            })
        
        frame_number += 1

    # Save metadata as a CSV
    metadata_json_path = os.path.join(output_dir, f"{video_filename}_metadata.json")
    with open(metadata_json_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    
    upload_to_blob(metadata_json_path, f"{BLOB_CONTAINER_NAME}-metadata")
    
    # Release resources
    cap.release()
    print(f"Frames and metadata have been saved in {output_dir}")


In [ ]:
# Process the video file
video_path = 'data/keynote_ms_build_2024.mp4'
output_dir = 'output_frames'
extract_frames(video_path, output_dir, interval=5)